# Transformer Based Language Model _(character level)_

Dataset - tiny shakespeare

In [1]:
# download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-09-24 19:29:05--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-09-24 19:29:05 (19.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



### Input

Understand and analyse the input

In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters: ", len(text))
# first 1000 characters
print(text[:1000])

length of dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hung

In [4]:
# unique chars in the datset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


build lookup tables and
tokenizer
- convert raw text(strings) to seq of integers

In practice, sub_words are used for tokenization, so for a sentence we will get only a few tokens

In [7]:
stoi_lookup = {char: i for i, char in enumerate(chars)}
itos_lookup = {i: char for i,char in enumerate(chars)}
# . already in the dataset so no need to add it

encode = lambda s: [stoi_lookup[char] for char in s]
decode = lambda ix: "".join(itos_lookup[i] for i in ix)

print(encode("rose by any name"))
print(decode(encode("rose by any name")))

[56, 53, 57, 43, 1, 40, 63, 1, 39, 52, 63, 1, 52, 39, 51, 43]
rose by any name


tokenise the dataset

In [8]:
import torch
tokenised_data = torch.tensor(encode(text), dtype=torch.long)
print(tokenised_data.shape, tokenised_data.dtype)
print(tokenised_data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

train, val split

In [11]:
train_w_end = int(0.9 * len(tokenised_data))
x_tr = tokenised_data[:train_w_end]
x_val = tokenised_data[train_w_end:]
print(x_tr.shape, x_val.shape)

torch.Size([1003854]) torch.Size([111540])


In [12]:
block_size = 8
x_tr[:block_size]+1

tensor([19, 48, 57, 58, 59,  2, 16, 48])

In [13]:
x = x_tr[:block_size]
y = x_tr[1:block_size+1]

for i in range(block_size):
  context = x[:i+1]
  target = y[i]
  print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [24]:
torch.manual_seed(1337)
batch_size = 4 # sequences for parallel processing
block_size = 8

def get_batch(split):
  # minibatch construct
  data = x_tr if split == 'train' else x_val
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i: i + block_size] for i in ix]) # stack rows
  y = torch.stack([data[i+1: i + (block_size+1)] for i in ix])
  return x, y


xb, yb = get_batch('train')
print('contexts:', xb.shape,)
print(xb, '\n')
print('targets:', yb.shape)
print(yb)

print('\n')

for b_i in range(batch_size):
  for ix in range(block_size):
    context = xb[b_i, :ix+1]
    target = yb[b_i, ix]
    print(f"for the input {context.tolist()} the target is: {target}")



contexts: torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]) 

targets: torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


for the input [24] the target is: 43
for the input [24, 43] the target is: 58
for the input [24, 43, 58] the target is: 5
for the input [24, 43, 58, 5] the target is: 57
for the input [24, 43, 58, 5, 57] the target is: 1
for the input [24, 43, 58, 5, 57, 1] the target is: 46
for the input [24, 43, 58, 5, 57, 1, 46] the target is: 43
for the input [24, 43, 58, 5, 57, 1, 46, 43] the target is: 39
for the input [44] the target is: 53
for the input [44, 53] the target is: 56
for the input [44, 53, 56] the target is: 1
for the input [44, 53, 56, 1] the target is: 58
for the input [44, 53, 56, 1, 58]

Bigram LM - uses counts for calculating the probs

In [54]:
import torch.nn as nn
from torch.nn import functional as F


torch.manual_seed(1337)

class BigramLM(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    # store the embeddings
    self.token_embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=vocab_size) # dict of len vocab_size with each value for vector with dim vocab_size

  def forward(self, i_xb, target=None):
    # forward a mini_batch of size batch_size with each tensor having context as the input

    # 1. caculcate the logits
    logits = self.token_embeddings(i_xb)              # B, T, C - batch, time, channel

    # print(logits.shape)

    if target is not None:
      B, T, C = logits.shape
      # 2. calculate the loss - nll/cross_entropy
      loss = F.cross_entropy(logits.view(B*T, C), target.view(B*T))            # cross_entropy only accepts ((C), (N,C) (N,C)), N - batch_size
    else:
      loss = None

    return logits, loss

  def generate(self, ix, max_new_tokens):
    # ix is (B, T) i.e (batch, time) => (4, 8)
    for _ in range(max_new_tokens):
      # get the targets
      logits, _ = self(ix)
      # what comes next in the sequence? the last char in the time dimension i.e context
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      # sampling for the next char from the dist
      next_ix = torch.multinomial(probs, num_samples=1)
      ix = torch.cat((ix, next_ix), dim=1) # running sampled indices for generating the next chars in the sequence
    return ix



bi_model = BigramLM(vocab_size)
logits, loss = bi_model(xb, yb)
print(xb.shape, yb.shape)
print(logits.shape)  # 4, 8, 65
print('loss is higher very high, ', loss.item() if loss > torch.log(torch.tensor(1/65)) else 'loss is ok')

ix = torch.zeros((1, 1), dtype=torch.long) # start from a newline char(0)
generated_ix = bi_model.generate(ix, max_new_tokens=100)
# print(generated_ix[0])
print('generated text: ', decode(generated_ix[0].tolist()))


torch.Size([32, 8]) torch.Size([32, 8])
torch.Size([32, 8, 65])
loss is higher very high,  4.794318199157715
generated text:  
Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


training the BigramML

In [55]:
optimizer = torch.optim.AdamW(bi_model.parameters(), lr=1e-3) # Adam instead of SGD, learning rate = 0.001

batch_size = 32

for step in range(20000):
  xb, yb = get_batch('train')
  # forward pass
  logits, loss = bi_model(xb, yb)
  # backward pass
  optimizer.zero_grad(set_to_none=True) # set grad= 0 from the prev steps as usual
  loss.backward()
  # update the model params
  optimizer.step()
  if step % 1000 == 0:
    print(f"step: {step}, loss: {loss.item()}")

print(loss.item())

step: 0, loss: 4.704006195068359
step: 1000, loss: 3.7031264305114746
step: 2000, loss: 3.1371781826019287
step: 3000, loss: 2.776794672012329
step: 4000, loss: 2.5844571590423584
step: 5000, loss: 2.5105180740356445
step: 6000, loss: 2.531585931777954
step: 7000, loss: 2.504757881164551
step: 8000, loss: 2.4696712493896484
step: 9000, loss: 2.4838879108428955
step: 10000, loss: 2.4183998107910156
step: 11000, loss: 2.529956817626953
step: 12000, loss: 2.379727363586426
step: 13000, loss: 2.4651365280151367
step: 14000, loss: 2.3533310890197754
step: 15000, loss: 2.4624433517456055
step: 16000, loss: 2.4509522914886475
step: 17000, loss: 2.325801372528076
step: 18000, loss: 2.4357123374938965
step: 19000, loss: 2.4936013221740723
2.4832067489624023


sampling from the model

In [57]:
ix = torch.zeros((1, 1), dtype=torch.long) # start from a newline char(0)
generated_ix = bi_model.generate(ix, max_new_tokens=500)
# print(generated_ix[0])
print('generated text: ', decode(generated_ix[0].tolist()))

generated text:  
fithods misue, knild he I:
Whe! toudirer' My ayosbly louroura s m', uthos s reveprthoukerdi't avorure fotemowe.
Whamo es t, tstt g t RTRushy,
WAsbr spr my ou pl y,
Witoft at o s me,
Whabr'the Cicuomants awonte qungur thme wrar d parsupl by:
'sul ve ave,
Kconit ped bim; fam elathelch easutlll teye A d che'd, met its

IVo wnkn cave!
I thengr ts, IO t
Hoyoolove
ONCENo breppo onder t this r is.
I ckend haiby Lulfaseabot alt mpw thar, d is cthe parn gers b, ace t d t tllined y sss u dst hothin h.
An,
